## 방법 2 . 질문을 “클러스터 ID”로 압축(과적합 더 줄이기)

In [1]:

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import ndcg_score

from xgboost import XGBRanker


In [2]:
df = pd.read_csv(".\\csv_파일\\3000개_질문.csv")

In [3]:
# llm_rank를 관련성 점수로 변환
# 1위 -> 5점, 2위 -> 3점, 3위 -> 2점, 나머지는 0점 (역순 변환)
def convert_rank_to_score(rank):
    if rank == 1: return 10
    elif rank == 2: return 5
    elif rank == 3: return 2
    else: return 0

# 정답(y)은 정수형으로 변환 (0~10 단계로 변환)
df['relevance_label'] = df['llm_rank'].apply(convert_rank_to_score)

---
### 데이터 분할

In [ ]:
# 1. 사용자 유형별로 질문(Group) 리스트 추출
queries_0 = df[df['사용자 유형'] == 0]['질문'].unique() # 입문자 질문들
queries_1 = df[df['사용자 유형'] == 1]['질문'].unique() # 기존 팬 질문들

# 2. 각 유형 내에서 8:2로 학습/테스트 질문 분리
# (전체 데이터에서도 8:2 비율이 유지되면서 테스트셋 구성됨)
train_q0, test_q0 = train_test_split(queries_0, test_size=0.2, random_state=42)
train_q1, test_q1 = train_test_split(queries_1, test_size=0.2, random_state=42)

# 3. 최종 학습 및 테스트 데이터 생성
train_df = df[df['질문'].isin(list(train_q0) + list(train_q1))].sort_values('질문').copy()
test_df  = df[df['질문'].isin(list(test_q0) + list(test_q1))].sort_values('질문').copy()

# 4) ✅ 인코더는 train에만 fit 하고, test는 transform만!
#    + test에 새로운 팀이 나와도 깨지지 않게 unknown_value=-1 처리
# OrdinalEncoder:
# 범주형(팀명)을 숫자로 바꾸는 인코더입니다.
# handle_unknown 옵션으로 test에 처음 보는 팀이 있어도 에러 대신 -1로 처리합니다.
enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

# train에서만 fit_transform: 팀명->숫자 매핑을 train로만 학습하고 변환합니다.
train_df[['기존_응원팀_encoded', '매칭팀_encoded']] = enc.fit_transform(
    train_df[['기존 응원 팀', '매칭팀']]
)

# test에서는 transform만: train에서 만든 매핑을 그대로 사용해서 숫자로 바꿉니다.
test_df[['기존_응원팀_encoded', '매칭팀_encoded']] = enc.transform(
    test_df[['기존 응원 팀', '매칭팀']]
)

# ============================================================
# 3) 질문 벡터화(TF-IDF) + 차원축소(SVD) (train에만 fit)
# ============================================================

# TF-IDF: 질문을 단어 중요도 벡터로 바꿉니다.
# ngram_range=(1,2): 단어(1-gram) + 단어쌍(2-gram)까지 반영합니다.
# min_df=2: 너무 희귀한 단어는 제거합니다.
# max_df=0.9: 너무 흔한 단어는 제거합니다.
tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df=2, max_df=0.9)

# train 질문으로만 fit_transform: 단어 사전/가중치 학습 + 변환
Xq_train_tfidf = tfidf.fit_transform(train_df['질문'])

# test 질문은 transform만: 학습은 절대 하지 않습니다(누수 방지)
Xq_test_tfidf = tfidf.transform(test_df['질문'])

# SVD: 고차원 TF-IDF 벡터를 저차원으로 압축합니다.
# n_components=50: 50차원으로 압축(초보자 기준 무난한 기본값)
svd = TruncatedSVD(n_components=50, random_state=42)

# train에서만 fit_transform: 압축 방향을 학습 + 변환
Xq_train_svd = svd.fit_transform(Xq_train_tfidf)

# test는 transform만: train에서 배운 압축 방향으로 변환
Xq_test_svd = svd.transform(Xq_test_tfidf)

# ============================================================
# 4) 질문 클러스터링(KMeans) → q_cluster 생성 (train에만 fit)
# ============================================================

# KMeans: 질문 벡터를 "주제(클러스터)"로 묶습니다.
# n_clusters=20: 질문 주제 20개로 압축(질문 수가 적으면 10~12로 낮추는 것도 추천)
kmeans = KMeans(n_clusters=20, random_state=42, n_init="auto")

# train 질문 벡터로만 학습하고, 각 행에 클러스터 번호를 부여합니다.
train_df['q_cluster'] = kmeans.fit_predict(Xq_train_svd)

# test 질문은 학습 없이, 어떤 클러스 터에 속하는지 예측만 합니다.
test_df['q_cluster'] = kmeans.predict(Xq_test_svd)

# ============================================================
# 5) 최종 feature 구성 + q_cluster One-Hot (연속값 오해 방지)
# ============================================================

# 기본 피처 목록(질문 원문은 포함하지 않습니다)
base_features = [
    '사용자 유형',
    'sbert_score',
    'n2v_score',
    'vector_score',
    '기존_응원팀_encoded',
    '매칭팀_encoded',
    'q_cluster'  # 질문을 "주제 ID"로 압축한 값
]

# train/test에서 동일한 피처 컬럼만 뽑아 X를 만듭니다.
X_train = train_df[base_features].copy()
X_test = test_df[base_features].copy()

# q_cluster는 "범주형"이므로 One-Hot으로 바꿉니다.
# (그냥 숫자로 두면 모델이 0<1<2 같은 '크기' 의미로 오해할 수 있음)
X_train = pd.get_dummies(X_train, columns=['q_cluster'], prefix='qcl')
X_test = pd.get_dummies(X_test, columns=['q_cluster'], prefix='qcl')

# train 기준 컬럼에 맞춰 test 컬럼을 정렬/보정합니다.
# (test에 없는 클러스터 컬럼은 0으로 채움)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# ============================================================
# 6) 라벨(y) 생성 (너가 쓰던 방식 유지)
# ============================================================

# llm_rank가 1~3이면 점수(3~1)로 변환, 그 외는 0
# - rank 1 -> 3점
# - rank 2 -> 2점
# - rank 3 -> 1점
# - 그 외 -> 0점
y_train = train_df['llm_rank'].apply(lambda x: 4 - x if x <= 3 else 0).astype(float)
y_test = test_df['llm_rank'].apply(lambda x: 4 - x if x <= 3 else 0).astype(float)

# ============================================================
# 7) 그룹(group) 계산 (질문별 후보 개수)
# ============================================================

# XGBRanker는 group이 "각 쿼리(질문)에 포함된 문서 수 리스트" 형태여야 합니다.
group_train = train_df.groupby('질문').size().to_list()
group_test = test_df.groupby('질문').size().to_list()

# 안전 점검: 그룹 합은 행 개수와 같아야 합니다.
assert sum(group_train) == len(train_df), "group_train 합이 train_df 행 수와 다릅니다."
assert sum(group_test) == len(test_df), "group_test 합이 test_df 행 수와 다릅니다."

# ============================================================


---
### 모델 학습

In [5]:
model = XGBRanker(
    objective="rank:ndcg",
    eval_metric="ndcg@3",
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

In [6]:
model.fit(X_train, y_train, group = group_train)

,objective,'rank:ndcg'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'ndcg@3'


🔍 LightGBM 로그 분석 결과
1. `No further splits with positive gain` (가장 많이 뜬 경고)
    - 의미: 나무를 더 잘게 쪼개서 학습하려고 했는데, 더 이상 쪼개봤자 성능 향상에 도움이 안 된다고 판단해서 멈춘 거야.

    - 원인: 우리 데이터(약 1,100행)가 머신러닝 기준으로는 아주 큰 편은 아니고, 피처(4개)가 명확해서 모델이 금방 정답 패턴을 찾았기 때문이야.

    - 결과: 이건 에러가 아니라 모델이 과적합(Overfitting)되지 않게 스스로 공부를 적당히 멈춘 똑똑한 상태라고 보면 돼!

2. `Found whitespace in feature_names`
    - 의미: 피처 이름 중에 '사용자 유형'처럼 공백(띄어쓰기)이 있는 게 있다는 뜻이야.

    - 결과: LightGBM이 알아서 사용자_유형처럼 언더바(_)로 바꿔서 처리했으니 아무 문제 없어.

3. `Found 'eval_at' in params`
    - 의미: eval_at 파라미터가 중복으로 설정됐다는 경고야.

    - 결과: 모델 생성할 때 넣은 설정값이 우선 적용됐다는 뜻이니 이것도 안심해도 돼.

In [7]:
scores = model.predict(X_test)

---
### 모델 검증 $\rightarrow$ 실제 추천 리스트로 변환

In [8]:
# 모델이 뱉은 점수를 test_df에 다시 붙여서, 질문별로 어떤 팀이 1등인지 눈으로 확인

# 1. 테스트 데이터 복사본에 예측 점수 추가
test_results = test_df.copy()
test_results['predicted_score'] = scores

# 2. 질문별로 점수가 높은 순(내림차순)으로 정렬
# '질문'과 'predicted_score' 순으로 정렬
test_results = test_results.sort_values(by=['질문', 'predicted_score'], ascending=[True, False])

# 3. 각 질문에서 모델이 뽑은 Top 3 팀 확인
final_top3 = test_results.groupby('질문').head(3)
print(final_top3[['질문', '매칭팀', 'predicted_score', 'llm_rank']])

# '사용자 유형'별로 필터링해서 저장해두면 나중에 피그마 UI에 넣기 좋아!
# final_top3.to_csv("final_recommendation_results.csv", index=False, encoding="utf-8-sig")

                                                     질문         매칭팀  \
2922          나는 EPL는 처음인데, 머니볼와(과) 압도적 성향이 강한 팀을 추천해줘.         페라리   
3729          나는 EPL는 처음인데, 머니볼와(과) 압도적 성향이 강한 팀을 추천해줘.         페라리   
2949          나는 EPL는 처음인데, 머니볼와(과) 압도적 성향이 강한 팀을 추천해줘.     삼성 라이온즈   
594                 나는 EPL는 처음인데, 신흥 강호 성향이 강한 팀을 추천해줘.  수원 삼성 블루윙즈   
591                 나는 EPL는 처음인데, 신흥 강호 성향이 강한 팀을 추천해줘.         페라리   
597                 나는 EPL는 처음인데, 신흥 강호 성향이 강한 팀을 추천해줘.   전북 현대 모터스   
2792           나는 EPL는 처음인데, 화끈와(과) 언더독 성향이 강한 팀을 추천해줘.         페라리   
3600           나는 EPL는 처음인데, 화끈와(과) 언더독 성향이 강한 팀을 추천해줘.         페라리   
3614           나는 EPL는 처음인데, 화끈와(과) 언더독 성향이 강한 팀을 추천해줘.     삼성 라이온즈   
3341                   나는 EPL는 처음인데, 효율 성향이 강한 팀을 추천해줘.         페라리   
2527                   나는 EPL는 처음인데, 효율 성향이 강한 팀을 추천해줘.         페라리   
2548                   나는 EPL는 처음인데, 효율 성향이 강한 팀을 추천해줘.     삼성 라이온즈   
1789                나는 F1는 처음인데, 우승권 유지 성향이 강한 팀을 추천해줘.         페라리   
1800  

### NDCG
: 추천된 팀들의 **순서(Ranking)**가 얼마나 정확한지 측정하는 지표.
</br> 1등 팀을 1등으로 맞추는 게 2등으로 맞추는 것보다 더 높은 점수를 받음.

- 0.7 이상: 모델이 꽤 영리하게 추천을 잘해줌

- 0.9 이상: LLM이 매긴 순위랑 거의 똑같이 맞추고 있음

- 만약 점수가 너무 낮게(0.5 이하) 나온다면?
</br>: 모델이 아직 '사용자 유형'이나 '벡터 스코어'의 특징을 제대로 못 잡았다는 신호
</br>$\Rightarrow$ learning_rate를 조절하거나 피처를 더 추가하는 파라미터 튜닝 단계 진행

In [9]:
# 모델 성능 평가하기 (NDCG 지표 확인)

from sklearn.metrics import ndcg_score
import numpy as np

# 1. 질문별 NDCG 점수를 저장할 리스트
ndcg_list = []

# 2. 질문(그룹)별로 루프를 돌며 계산
for qid in test_results['질문'].unique():
    # 해당 질문에 속한 데이터만 추출
    group_data = test_results[test_results['질문'] == qid]
    
    # 실제 정답(llm_rank 기반 점수)과 모델의 예측 점수 추출
    # ndcg_score는 2차원 배열 형식을 원하므로 reshape(1, -1)을 해줘.
    y_true = np.array([group_data['relevance_label']]).reshape(1, -1)
    y_score = np.array([group_data['predicted_score']]).reshape(1, -1)
    
    # NDCG@3 계산 (상위 3개 팀의 순위가 얼마나 정확한가?)
    # 데이터가 3개보다 적을 경우를 대비해 k값을 조절해.
    k = min(len(group_data), 3)
    score = ndcg_score(y_true, y_score, k=k)
    ndcg_list.append(score)

# 3. 최종 평균 NDCG 출력
avg_ndcg = np.mean(ndcg_list)
print(f" 최종 모델 성능 (Average NDCG@3): {avg_ndcg:.4f}")


 최종 모델 성능 (Average NDCG@3): 0.5486


In [10]:
from itertools import product
from xgboost import XGBRanker
from sklearn.metrics import ndcg_score
import numpy as np

param_grid = {
    "learning_rate": [0.05, 0.1],
    "max_depth": [4, 6],
    "n_estimators": [200, 400],
    "subsample": [0.8, 1.0]
}

best_score = -1
best_params = None

for lr, md, ne, ss in product(
    param_grid["learning_rate"],
    param_grid["max_depth"],
    param_grid["n_estimators"],
    param_grid["subsample"]
):
    model = XGBRanker(
        objective="rank:ndcg",
        learning_rate=lr,
        max_depth=md,
        n_estimators=ne,
        subsample=ss,
        random_state=42
    )

    model.fit(
        X_train, y_train,
        group = group_train
    )

    preds = model.predict(X_test)

    score = ndcg_score(
        [y_test],
        [preds]
    )

    if score > best_score:
        best_score = score
        best_params = {
            "learning_rate": lr,
            "max_depth": md,
            "n_estimators": ne,
            "subsample": ss
        }

print("Best NDCG:", best_score)
print("Best Params:", best_params)


Best NDCG: 0.8780360148071866
Best Params: {'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 400, 'subsample': 0.8}


## MinMaxScaler 전체 코드

In [5]:
# ============================================================
# 방법 2 + MinMaxScaler 전체 코드
# ============================================================

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import ndcg_score

from xgboost import XGBRanker

df = pd.read_csv(".\\csv_파일\\16000개_데이터.csv")

# ============================================================
# 1) 질문 단위 train / test 분리
# ============================================================

queries_0 = df[df['사용자 유형'] == 0]['질문'].unique()
queries_1 = df[df['사용자 유형'] == 1]['질문'].unique()

train_q0, test_q0 = train_test_split(queries_0, test_size=0.2, random_state=42)
train_q1, test_q1 = train_test_split(queries_1, test_size=0.2, random_state=42)

train_df = df[df['질문'].isin(list(train_q0) + list(train_q1))].sort_values('질문').copy()
test_df  = df[df['질문'].isin(list(test_q0)  + list(test_q1))].sort_values('질문').copy()

# ============================================================
# 2) 팀 인코딩 (OrdinalEncoder, train에만 fit)
# ============================================================

enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

train_df[['기존_응원팀_encoded', '매칭팀_encoded']] = enc.fit_transform(
    train_df[['기존 응원 팀', '매칭팀']]
)

test_df[['기존_응원팀_encoded', '매칭팀_encoded']] = enc.transform(
    test_df[['기존 응원 팀', '매칭팀']]
)

# ============================================================
# 3) 질문 벡터화 + 차원 축소 (TF-IDF + SVD)
# ============================================================

tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df=2, max_df=0.9)

Xq_train_tfidf = tfidf.fit_transform(train_df['질문'])
Xq_test_tfidf  = tfidf.transform(test_df['질문'])

svd = TruncatedSVD(n_components=50, random_state=42)

Xq_train_svd = svd.fit_transform(Xq_train_tfidf)
Xq_test_svd  = svd.transform(Xq_test_tfidf)

# ============================================================
# 4) 질문 클러스터링 → q_cluster
# ============================================================

kmeans = KMeans(n_clusters=20, random_state=42, n_init="auto")

train_df['q_cluster'] = kmeans.fit_predict(Xq_train_svd)
test_df['q_cluster']  = kmeans.predict(Xq_test_svd)

# ============================================================
# 5) feature 구성 + One-Hot
# ============================================================

base_features = [
    '사용자 유형',
    'sbert_score',
    'n2v_score',
    'vector_score',
    '기존_응원팀_encoded',
    '매칭팀_encoded',
    'q_cluster'
]

X_train = train_df[base_features].copy()
X_test  = test_df[base_features].copy()

# q_cluster One-Hot
X_train = pd.get_dummies(X_train, columns=['q_cluster'], prefix='qcl')
X_test  = pd.get_dummies(X_test, columns=['q_cluster'], prefix='qcl')

# 컬럼 정렬
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# ============================================================
# 6) ✅ MinMaxScaler 적용 (train에만 fit)
# ============================================================

scaler = MinMaxScaler()

# pandas → numpy 변환 후 스케일링 → 다시 DataFrame
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)

# ============================================================
# 7) 라벨 생성
# ============================================================

y_train = train_df['llm_rank'].apply(lambda x: 4 - x if x <= 3 else 0).astype(float)
y_test  = test_df['llm_rank'].apply(lambda x: 4 - x if x <= 3 else 0).astype(float)

# ============================================================
# 8) 그룹 정보
# ============================================================

group_train = train_df.groupby('질문').size().to_list()
group_test  = test_df.groupby('질문').size().to_list()

assert sum(group_train) == len(train_df)
assert sum(group_test)  == len(test_df)

# ============================================================
# 9) XGBRanker 학습
# ============================================================

model = XGBRanker(
    objective="rank:ndcg",
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=42
)

model.fit(X_train_scaled, y_train, group=group_train)

# ============================================================
# 10) 예측 + NDCG 평가
# ============================================================

pred_test = model.predict(X_test_scaled)

def mean_ndcg_by_group(y_true, y_pred, group_sizes, k=5):
    scores = []
    start = 0
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    for g in group_sizes:
        end = start + g
        kk = min(k, g)
        scores.append(
            ndcg_score(
                y_true[start:end].reshape(1, -1),
                y_pred[start:end].reshape(1, -1),
                k=kk
            )
        )
        start = end
    return float(np.mean(scores))

print("NDCG@3 :", mean_ndcg_by_group(y_test, pred_test, group_test, k=3))
print("NDCG@5 :", mean_ndcg_by_group(y_test, pred_test, group_test, k=5))
print("NDCG@10:", mean_ndcg_by_group(y_test, pred_test, group_test, k=10))

NDCG@3 : 0.719814486436245
NDCG@5 : 0.7005150914105156
NDCG@10: 0.6684445276155954


In [6]:
from itertools import product
from xgboost import XGBRanker
from sklearn.metrics import ndcg_score
import numpy as np

param_grid = {
    "learning_rate": [0.05, 0.1],
    "max_depth": [4, 6],
    "n_estimators": [200, 400],
    "subsample": [0.8, 1.0]
}

best_score = -1
best_params = None

for lr, md, ne, ss in product(
    param_grid["learning_rate"],
    param_grid["max_depth"],
    param_grid["n_estimators"],
    param_grid["subsample"]
):
    model = XGBRanker(
        objective="rank:ndcg",
        learning_rate=lr,
        max_depth=md,
        n_estimators=ne,
        subsample=ss,
        random_state=42
    )

    model.fit(
        X_train, y_train,
        group = group_train
    )

    preds = model.predict(X_test)

    score = ndcg_score(
        [y_test],
        [preds]
    )

    if score > best_score:
        best_score = score
        best_params = {
            "learning_rate": lr,
            "max_depth": md,
            "n_estimators": ne,
            "subsample": ss
        }

print("Best NDCG:", best_score)
print("Best Params:", best_params)


Best NDCG: 0.905098755697025
Best Params: {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 400, 'subsample': 0.8}
